In [1]:
# Import libraries
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from google.colab import files

# List of categorised fault types
fault_types = [
    'Power-related', 'Communication', 'Sensor-related',
    'Control system', 'Environmental', 'Cybersecurity'
]

# List of categorised severity levels
severity_levels = ['Low', 'Medium', 'High', 'Critical', 'Informational']

# List of fault statuses representing different stages
fault_statuses = ['Detected', 'Acknowledged', 'In Progress', 'Resolved', 'Closed']

# List of monitored locations
locations = [
    'City Hall', 'Outram Park', 'Pasir Ris', 'Jurong West',
    'Orchard Road', 'Toa Payoh', 'Canberra', 'Bugis Junction',
    'Bishan', 'Chinatown'
]

# Generate light ids and assign random locations
def generate_light_ids(num_lights):  # Function takes the total number of lights to generate
    light_ids = []  # Create empty list to store ids
    light_locations = {}  # Create empty dictionary to map each light id to its location

    # Loop for each light
    for i in range(1, num_lights + 1): # For each light from 1 to number of lights
        light_id = f'L{i:04d}'  # Create id: L0001
        location_name = random.choice(locations)  # Pick a random location from the locations list
        light_ids.append(light_id)  # Add the generated id to the list
        light_locations[light_id] = {'location_name': location_name}  # Store the location info keyed by light id
    return light_ids, light_locations  # Return both the list of ids and maps each light id to its location

# Generate a random number of fault records
def generate_fault_records(light_ids, light_locations, start_date, end_date,  min_records=100, max_records=500): # Function will pick a random total faults between 100 and 500
    num_records = random.randint(min_records, max_records) # Random total records count
    records = []
    total_seconds = (end_date - start_date).total_seconds() # Total seconds between dates

    for _ in range(num_records):
        # Randomly pick a light and get its location
        light_id = random.choice(light_ids) # Pick a random light_id from light_ids
        location_name = light_locations[light_id]['location_name'] # Get location information for that light

        # Randomly select fault details
        fault_type = random.choice(fault_types) # Pick a random fault type
        severity_level = random.choice(severity_levels) # Pick a random severity level
        fault_status = random.choice(fault_statuses) # Pick a random fault status

        # Generate a random timestamp between start and end date, rounded to seconds
        random_seconds = random.uniform(0, total_seconds) # Pick a random seconds within the range, random.uniform(0, total_seconds) gives a random amount of time (in seconds) between the start and end dates
        timestamp = (start_date + timedelta(seconds=random_seconds)).replace(microsecond=0) # Add random seconds to start date

        # Assign a maintenance cost (90% chance to have a value, 10% chance missing value)
        if random.random() > 0.1:  # 90% of the time
            maintenance_cost = round(random.uniform(50, 500), 2)  # Cost between $50 and $500
        else:  # 10% of the time
            maintenance_cost = np.nan  # Missing value

         # Create a dictionary representing one fault record
        record = {
            'light_id': light_id,  # Unique identifier of the street light
            'location_name': location_name, # Name of the light's location
            'fault_type': fault_type, # Current fault type from the list
            'timestamp': timestamp, # Time when the fault was recorded
            'severity_level': severity_level,  # Severity level of the fault
            'fault_status': fault_status, # Current status of the fault
            'maintenance_cost': maintenance_cost  # Estimated cost to fix the fault
        }

        records.append(record) # Add the fault record to the list of records
    # Return the list containing one record per fault type
    return records

# Generate fault data
def generate_data(num_lights=1000, min_faults=100, max_faults=500,
                  start_date=datetime(2020, 1, 1),
                  end_date=datetime(2020, 12, 31),
                  save_csv=True,
                  filename='lighting_data.csv'):

    # Display how many records will be generated
    print(f"Generating between {min_faults} and {max_faults} fault records for {num_lights} lights.")

    # 1. Generate light ids and random locations for each light
    light_ids, light_locations = generate_light_ids(num_lights)

    # 2. Generate random fault records for the lights
    data = generate_fault_records(light_ids, light_locations, start_date, end_date, min_faults, max_faults)

    # 3. Convert list of fault records into a pandas dataframe
    df = pd.DataFrame(data)  # Convert list of records into a table
    df = df.sort_values(by='timestamp').reset_index(drop=True) # Sort table by time

    # 4. Save to csv and download
    if save_csv:
        df.to_csv(filename, index=False) # Save dataframe to csv
        files.download(filename) # Automatically download the file in colab
    return df

# Run data colab
df = generate_data()
df

Generating between 100 and 500 fault records for 1000 lights.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,light_id,location_name,fault_type,timestamp,severity_level,fault_status,maintenance_cost
0,L0313,Jurong West,Power-related,2020-01-07 09:05:35,Low,Closed,297.45
1,L0385,Pasir Ris,Environmental,2020-01-07 10:28:12,Medium,In Progress,172.61
2,L0158,Canberra,Communication,2020-01-07 13:09:56,Medium,Detected,381.23
3,L0554,Jurong West,Communication,2020-01-08 01:55:51,Medium,In Progress,284.30
4,L0755,Bishan,Communication,2020-01-11 10:49:07,Critical,In Progress,76.52
...,...,...,...,...,...,...,...
163,L0768,Chinatown,Cybersecurity,2020-12-24 21:14:30,Critical,Detected,156.92
164,L0716,Pasir Ris,Control system,2020-12-26 03:30:47,High,Closed,NaN
165,L0085,Toa Payoh,Communication,2020-12-27 02:25:18,Medium,Closed,377.93
166,L0525,Pasir Ris,Control system,2020-12-28 00:39:48,Low,In Progress,191.01
